## Load in the expression file

In [1]:
#set working directory
setwd("./One_expression_file_example/");

In [2]:
 Human_Proteome_Map_spectral_count_gene_tissue <- read.table("Human_Proteome_Map_spectral_count_gene_tissue.txt",header=TRUE, 
                                                             sep = "\t",as.is = TRUE, quote="\"")

In [3]:
path_to_file="/Users/risserlin/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/Mastermap/notebooks/One_expression_file_example/"
exp_file="/Users/risserlin/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/Mastermap/notebooks/One_expression_file_example/Human_Proteome_Map_spectral_count_gene_tissue.txt"
#on windows change the \ to / in order for the pathname to be interpretted correctly.
#exp_file="C:/Users/zaphod/Ruth_dropbox/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/Mastermap/notebooks/One_expression_file_example/Human_Proteome_Map_spectral_count_gene_tissue.txt"
#path_to_file="C:/Users/zaphod/Ruth_dropbox/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/Mastermap/notebooks/One_expression_file_example/"

## function to calculate ranked genelist using KS test

In [4]:
dataKSLGtest_stats = function(data, num_samples = dim(data)[2], end = dim(data)[2]/2 ){

#given a matrix with a set of points for two different states
# calculate the p-values for each pair of states using a ks-test.  Both the lesser
# and greater KS test is calculated.  Whichever statistic is the largest is returned.
# This method by default assumes that there are equal data points for each
# state so it get the column count of x and assumes that the first
# x/2 columns are from state 1 and the remaining are from state 2.


#if there is not an equal number of state 1 and state 2 the user can specify
# the number of data points and how you want to divide them by telling it how 
# many samples should be in the first group
     
    #create a matrix to store the resulting pvalues and statistics
    stat.mat = matrix(0,dim(data)[1],2)

    num_iter = dim(data)[1]
    start = end + 1

    for(i in 1:num_iter){

            less_temp = try(suppressWarnings(ks.test(t(data[i,1:end]), t(data[i,start:num_samples]),alternative = "less")),TRUE)
            if(inherits(less_temp, "try-error")) stat.mat[i,1] = 1

            greater_temp = try(suppressWarnings(ks.test(t(data[i,1:end]), t(data[i,start:num_samples]),alternative = "greater")),TRUE)
            if(inherits(greater_temp, "try-error")) stat.mat[i,1] = 1

            if(less_temp$statistic > greater_temp$statistic){
                stat.mat[i,1] = less_temp$statistic
                stat.mat[i,2] = less_temp$p.value
            }
            else if(less_temp$statistic <= greater_temp$statistic){
                stat.mat[i,1] = (-1) * greater_temp$statistic
                stat.mat[i,2] = greater_temp$p.value
            }

    }                
    stat.mat

}

In [17]:
library("gProfileR");

#calculate the ks test statistics for each column 3 - 32

#create a temporary matrix
for(j in 3:32){
    if(j==3){
      current <- Human_Proteome_Map_spectral_count_gene_tissue[,3:32]
      rownames(current) <- rownames(Human_Proteome_Map_spectral_count_gene_tissue)
      colnames(current) <- colnames(Human_Proteome_Map_spectral_count_gene_tissue)[3:32]
    }
    else{
        current <- cbind(Human_Proteome_Map_spectral_count_gene_tissue[,j:32],Human_Proteome_Map_spectral_count_gene_tissue[,3:(j-1)])
        rownames(current) <- rownames(Human_Proteome_Map_spectral_count_gene_tissue)
        colnames(current) <- c(colnames(Human_Proteome_Map_spectral_count_gene_tissue)[j:32], colnames(Human_Proteome_Map_spectral_count_gene_tissue)[3:(j-1)])
    }
    ks_stats_current <- dataKSLGtest_stats(current, end = 1)
    rownames(ks_stats_current) <-  Human_Proteome_Map_spectral_count_gene_tissue[,1]
    
    #get the set of genes that are in the top 5% by p-value to be tissue specific
    top_5percentile <- quantile(ks_stats_current[,2],0.05)
    genes <- rownames(ks_stats_current)[which(ks_stats_current[,2]<top_5percentile & ks_stats_current[,1] >0)]
    
    #order the genes by ks statistic and only include the genes that have positive statistic
    a <- ks_stats_current[which(ks_stats_current[,1] >0),]
    genes_ordered <- a[order(a[,1],decreasing = TRUE),]
    
    
    #run gprofiler on the set of genes 
    gprofiler_results <- gprofiler(rownames(genes_ordered),significant=F,ordered_query = T,
                                           exclude_iea=T,max_set_size = 500,
                                           correction_method = "fdr",
                                           src_filter = c("GO:BP","KEGG","REAC"))
    
    #filter gprofiler results
    gprofiler_results <- gprofiler_results[which(gprofiler_results[,'term.size'] >= 3
                                        & gprofiler_results[,'overlap.size'] >= 2 ),]
    if(dim(gprofiler_results)[1] > 1){
        #write out the gprofiler results
        # gProfileR returns corrected p-values only.  Set p-value to corrected p-value
        em_results <- cbind(gprofiler_results[,c("term.id","term.name","p.value","p.value")], 1,
                                gprofiler_results[,"intersection"])
        colnames(em_results) <- c("Name","Description", "pvalue","qvalue","phenotype","genes")

        em_results_filename <- paste("./gprofiler/",colnames(current)[1],"_gprofiler_results_ordered_nothresh.txt",sep="")
        em_results_filename_fullpath <- paste(path_to_file,"/gprofiler/",colnames(current)[1],"_gprofiler_results.txt",sep="")

        write.table(em_results,em_results_filename,col.name=TRUE,sep="\t",row.names=FALSE,quote=FALSE)
    }
}

In [7]:
j <- 3
current <- Human_Proteome_Map_spectral_count_gene_tissue[,3:32]
      rownames(current) <- rownames(Human_Proteome_Map_spectral_count_gene_tissue)
      colnames(current) <- colnames(Human_Proteome_Map_spectral_count_gene_tissue)[3:32]

ks_stats_current <- dataKSLGtest_stats(current, end = 1)
    rownames(ks_stats_current) <-  Human_Proteome_Map_spectral_count_gene_tissue[,1]
    
rownames(ks_stats_current) <- Human_Proteome_Map_spectral_count_gene_tissue[,1]
min(ks_stats_current[,2])
top_5percentile <- quantile(ks_stats_current[,2],0.05)
genes <- rownames(ks_stats_current)[which(ks_stats_current[,2]<top_5percentile & ks_stats_current[,1] >0)]

[1] 0.1446652

In [15]:
dim(b)

[1] 5371    2

In [31]:
library("gProfileR");

gprofiler_results <- gprofiler(genes,significant=T,ordered_query = F,
                                           exclude_iea=T,max_set_size = 500,
                                           correction_method = "fdr",
                                           src_filter = c("GO:BP","KEGG","REAC"))

In [32]:
head(gprofiler_results)

,query.number,significant,p.value,term.size,query.size,overlap.size,recall,precision,term.id,domain,subgraph.number,term.name,relative.depth,intersection
1,1,TRUE,0.000221,432,428,28,0.065,0.065,GO:0007017,BP,70,microtubule-based process,1,"STMN4,EPHA3,CDC42,CLASP1,DYNC1I2,MAPRE1,STMN2,ARHGAP21,APC2,KIF21B,STMN1,TUBA1B,DYNLRB1,KLC1,MACF1,FGF13,SPIRE1,APC,DYNLT1,KIF5A,TUBA1A,TUBA1C,KIF5C,CHD3,CKAP5,TUBB,DYNC1H1,DYX1C1"
2,1,TRUE,0.0291,189,428,13,0.03,0.069,GO:0032886,BP,70,regulation of microtubule-based process,2,"EPHA3,CDC42,CLASP1,MAPRE1,STMN2,APC2,STMN1,MACF1,FGF13,APC,CHD3,CKAP5,DYNC1H1"
3,1,TRUE,0.0318,168,428,12,0.028,0.071,GO:0070507,BP,70,regulation of microtubule cytoskeleton organization,1,"EPHA3,CDC42,CLASP1,MAPRE1,STMN2,APC2,STMN1,FGF13,APC,CHD3,CKAP5,DYNC1H1"
4,1,TRUE,0.000256,13,428,5,0.012,0.385,GO:0008045,BP,37,motor neuron axon guidance,1,"PLXNA2,PLXNA1,PLXNA3,PLXNC1,PLXNA4"
5,1,TRUE,1.39e-06,6,428,5,0.012,0.833,GO:0021785,BP,37,branchiomotor neuron axon guidance,2,"PLXNA2,PLXNA1,PLXNA3,PLXNC1,PLXNA4"
6,1,TRUE,0.0124,414,428,23,0.054,0.056,GO:0010563,BP,49,negative regulation of phosphorus metabolic process,1,"PRKAR2B,GNAI3,YWHAE,OPRM1,LRRC4,LATS1,LRRC4B,APC,PPP1R1A,LMTK3,TIPRL,LRRC4C,LATS2,ADRA2A,PPP1R1C,NELL1,YWHAB,YWHAG,PKIA,SH3BP5L,PPM1E,SFN,GRM3"


In [33]:
gprofiler_results <- gprofiler_results[which(gprofiler_results[,'term.size'] >= 3
                                        & gprofiler_results[,'overlap.size'] >= 2 ),]

In [50]:
# gProfileR returns corrected p-values only.  Set p-value to corrected p-value
em_results <- cbind(gprofiler_results[,c("term.id","term.name","p.value","p.value")], 1,
                                gprofiler_results[,"intersection"])
colnames(em_results) <- c("Name","Description", "pvalue","qvalue","phenotype","genes")

em_results_filename <- paste("./gprofiler/",colnames(current)[1],"_gprofiler_results.txt",sep="")
em_results_filename_fullpath <- paste(path_to_file,"/gprofiler/",colnames(current)[1],"_gprofiler_results.txt",sep="")

write.table(em_results,em_results_filename,col.name=TRUE,sep="\t",row.names=FALSE,quote=FALSE)

In [36]:
library(RJSONIO)

library(httr)
# Basic settings
port.number = 1234
base.url = paste("http://localhost:", toString(port.number), "/v1", sep="")

print(base.url)

version.url = paste(base.url, "version", sep="/")
cytoscape.version = GET(version.url)
cy.version = fromJSON(rawToChar(cytoscape.version$content))
print(cy.version)

[1] "http://localhost:1234/v1"
      apiVersion cytoscapeVersion 
            "v1"          "3.4.0" 


In [51]:
# to create an Enrichment map we need to specify
# analysisType = generic
# 
enrichmentmap.url <- paste(base.url, "commands","enrichmentmap","build", sep="/") 

em_params <- list(analysisType = "generic",enrichmentsDataset1 = em_results_filename_fullpath,pvalue="1.0",qvalue="0.00001",
                  expressionDataset1 = exp_file, 
                  similaritycutoff="0.25",coeffecients="JACCARD")

response <- GET(url=enrichmentmap.url, query=em_params)

In [52]:
content(response, "text", encoding = "ISO-8859-1")

[1] "Finished\n"